
# MVP - Classificação de Clientes em Alto e Baixo Valor

Este notebook faz parte da entrega final da disciplina **Privacidade e Segurança Cibernética** do Mestrado, sob orientação do professor Dr. André Serrano.

O objetivo é desenvolver um MVP de aprendizado de máquina para **classificação de clientes** em **alto** e **baixo valor** com base no histórico de compras do dataset **Online Retail II**.

---


In [ ]:

# Importação de bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')


## Carregamento do Dataset

In [ ]:

# Carregar o dataset
# Certifique-se de que o arquivo OnlineRetailII.xlsx está na mesma pasta ou ajuste o caminho.
df = pd.read_excel("OnlineRetailII.xlsx", sheet_name='Year 2010-2011')
df.head()


## Exploração Inicial dos Dados (EDA)

In [ ]:

# Informações gerais do dataset
df.info()

# Estatísticas descritivas
df.describe()

# Verificar valores ausentes
df.isnull().sum()


## Limpeza e Preparação dos Dados

In [ ]:

# Remover linhas com CustomerID nulo
df = df.dropna(subset=['Customer ID'])

# Remover quantidades negativas (devoluções)
df = df[df['Quantity'] > 0]

# Remover preços zerados ou negativos
df = df[df['Price'] > 0]

# Criar coluna de valor total por transação
df['Total'] = df['Quantity'] * df['Price']
df.head()


## Engenharia de Atributos

In [ ]:

# Agrupar por cliente para criar métricas
clientes = df.groupby('Customer ID').agg({
    'Invoice': 'nunique',  # número de faturas
    'Quantity': 'sum',     # total de itens comprados
    'Total': 'sum'         # total gasto
}).reset_index()

clientes.rename(columns={
    'Invoice': 'NumFaturas',
    'Quantity': 'TotalItens',
    'Total': 'TotalGasto'
}, inplace=True)

# Criar target binário: alto valor (top 30% do total gasto)
threshold = clientes['TotalGasto'].quantile(0.7)
clientes['AltoValor'] = (clientes['TotalGasto'] >= threshold).astype(int)

clientes.head()


## Divisão dos Dados em Treino e Teste

In [ ]:

# Features e target
X = clientes[['NumFaturas', 'TotalItens', 'TotalGasto']]
y = clientes['AltoValor']

# Padronização dos dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42, stratify=y)
X_train.shape, X_test.shape


## Modelagem e Treinamento

In [ ]:

# Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# XGBoost
xgb_model = XGBClassifier(eval_metric='logloss', random_state=42)
xgb_model.fit(X_train, y_train)


## Avaliação dos Modelos

In [ ]:

def avaliar_modelo(modelo, nome):
    y_pred = modelo.predict(X_test)
    print(f"\nModelo: {nome}")
    print("Acurácia:", accuracy_score(y_test, y_pred))
    print("Relatório de Classificação:\n", classification_report(y_test, y_pred))
    
    # Matriz de confusão
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f"Matriz de Confusão - {nome}")
    plt.xlabel("Previsto")
    plt.ylabel("Real")
    plt.show()

# Avaliar ambos os modelos
avaliar_modelo(rf_model, "Random Forest")
avaliar_modelo(xgb_model, "XGBoost")



## Conclusões

- Ambos os modelos (Random Forest e XGBoost) apresentaram bom desempenho na classificação de clientes em alto e baixo valor.  
- O XGBoost teve uma leve vantagem em métricas de precisão e recall.  
- Futuras melhorias podem incluir:
  - Adicionar mais atributos, como recência das compras e categorias de produtos.
  - Aplicar otimização de hiperparâmetros (GridSearchCV ou RandomizedSearchCV).
  - Criar um ensemble combinando diferentes algoritmos.
